In [1]:
import pandas as pd

In [ ]:
tarifas = pd.read_excel('./dados/comex/impacto_empresas/tarifas_totais_232.xlsx')

In [104]:
empresas = pd.read_excel('./dados/comex/impacto_empresas/Diagnóstico Impacto do Tarifaço EUA sobre as Empresas Brasileiras(1-218).xlsx',dtype={'Liste os Códigos Tarifários (NCM) mais Representativos nas Exportações para os EUA:': str})

In [105]:
empresas.shape

(218, 12)

In [106]:
#Criar uma coluna ncm que é o resultado da remoção de caracteres diferentes de números, pontos, espaços \n e tab da coluna `Liste os Códigos Tarifários (NCM) mais Representativos nas Exportações para os EUA:``

empresas['ncm'] = empresas['Liste os Códigos Tarifários (NCM) mais Representativos nas Exportações para os EUA:'].str.replace(r'[^\d \n\t]', '', regex=True)


In [107]:
empresas[['Liste os Códigos Tarifários (NCM) mais Representativos nas Exportações para os EUA:', 'ncm']]

,Liste os Códigos Tarifários (NCM) mais Representativos nas Exportações para os EUA:,ncm
0,47032900,47032900
1,"48102990, 48109290, 48026199\n",48102990 48109290 48026199\n
2,7202.30.00\n7202.11.00\n,72023000\n72021100\n
3,7202.21.00 \n2804.69.00,72022100 \n28046900
4,MDF - 441112 / 441113 / 441114 \nMDP - 441011,441112 441113 441114 \n 441011
...,...,...
213,4114001,4114001
214,19063210,19063210
215,93063000\n82055900\n56031340\n48059100\n701971...,93063000\n82055900\n56031340\n48059100\n701971...
216,3004.49.20\n2106.90.90\n1302.19.99,30044920\n21069090\n13021999


In [108]:
#Separar o ncm em uma lista
empresas['ncm'] = empresas['ncm'].str.split()

In [109]:
empresas.head(2)

,ID,Start time,Completion time,Email,Name,Last modified time,Empresa,"Contato (Nome, Telefone e E-mail)",Qual a participação (%) das exportações aos Estados Unidos na receita total da empresa?\n(Resposta numérica – ex: 25),Qual a participação (%) das exportações aos Estados Unidos em relação ao total exportado pela empresa?\n(Resposta numérica – ex: 25)\n,"Quais mercados a empresa entende que podem ser acessados no médio prazo?\n(Resposta longa – ex: América Latina, Oriente Médio, Sudeste Asiático etc.)\n",Liste os Códigos Tarifários (NCM) mais Representativos nas Exportações para os EUA:,ncm
0,1,2025-07-24 13:26:12,2025-07-24 13:30:37,anonymous,NaN,NaN,Celulose Nipo-Brasileira S/A - CENIBRA,Júlio César Tôrres Ribeiro\n+55-31-99126-8778\...,"10,5","9,3","Ásia e Europa, porém são mercados maduros e de...",47032900,[47032900]
1,2,2025-07-24 14:34:06,2025-07-24 14:38:36,anonymous,NaN,NaN,Ibema Companhia Brasileira de Papel,"Fernando Carvalho, (41)99201-8182, fernando.ca...",5%,15%,"Latam, África e Europa","48102990, 48109290, 48026199\n","[48102990, 48109290, 48026199]"


In [110]:
#Explodir o dataframe criando um registro para cada linha de ncm
empresas = empresas.explode('ncm')


In [111]:
# 4. Remover valores nulos após explode
empresas = empresas.dropna(subset=['ncm'])
#remover valores de ncm com menos de 2 dígitos
empresas = empresas[empresas['ncm'].str.len() >= 2]


In [112]:
# 5. Tratar códigos NCM individuais (não listas)
empresas['ncm'] = empresas['ncm'].apply(lambda x: str(x)[:6] if len(str(x)) > 6 else str(x))


In [113]:
empresas.query('ncm.str.len() != 2 and ncm.str.len() != 4 and ncm.str.len() != 6', engine='python')

,ID,Start time,Completion time,Email,Name,Last modified time,Empresa,"Contato (Nome, Telefone e E-mail)",Qual a participação (%) das exportações aos Estados Unidos na receita total da empresa?\n(Resposta numérica – ex: 25),Qual a participação (%) das exportações aos Estados Unidos em relação ao total exportado pela empresa?\n(Resposta numérica – ex: 25)\n,"Quais mercados a empresa entende que podem ser acessados no médio prazo?\n(Resposta longa – ex: América Latina, Oriente Médio, Sudeste Asiático etc.)\n",Liste os Códigos Tarifários (NCM) mais Representativos nas Exportações para os EUA:,ncm
84,85,2025-07-28 21:42:43,2025-07-28 22:11:48,anonymous,NaN,NaN,MBRF,Thalita Bento de Souza/ (11) 94274-2549 / tha...,4%,6%,"Vietnã (bovino); Indonésia (Suino); México, Ca...",0201.30.00; 0202.3000; 0203.2; 1602.50.00,02032


In [114]:
empresas_2d = empresas.query('ncm.str.len() == 2', engine='python')
empresas_6d = empresas.query('ncm.str.len() == 6', engine='python')
empresas_4d = empresas.query('ncm.str.len() == 4', engine='python')
empresas_5d = empresas.query('ncm.str.len() == 5', engine='python')

In [115]:
print(f'Tamanhos dos dataframes são 2d: {empresas_2d.shape[0]}, 4d: {empresas_4d.shape[0]}, 5d: {empresas_5d.shape[0]}, 6d: {empresas_6d.shape[0]} e a soma é {empresas_2d.shape[0] + empresas_4d.shape[0] + empresas_5d.shape[0] + empresas_6d.shape[0]}  ')

Tamanhos dos dataframes são 2d: 1, 4d: 15, 5d: 1, 6d: 543 e a soma é 560  


In [130]:
tarifas['SH6'] = tarifas['SH6'].astype(str)
tarifas['SH6'] = tarifas['SH6'].str.zfill(6)

In [122]:
empresas_5d.head(2)

,ID,Start time,Completion time,Email,Name,Last modified time,Empresa,"Contato (Nome, Telefone e E-mail)",Qual a participação (%) das exportações aos Estados Unidos na receita total da empresa?\n(Resposta numérica – ex: 25),Qual a participação (%) das exportações aos Estados Unidos em relação ao total exportado pela empresa?\n(Resposta numérica – ex: 25)\n,"Quais mercados a empresa entende que podem ser acessados no médio prazo?\n(Resposta longa – ex: América Latina, Oriente Médio, Sudeste Asiático etc.)\n",Liste os Códigos Tarifários (NCM) mais Representativos nas Exportações para os EUA:,ncm
84,85,2025-07-28 21:42:43,2025-07-28 22:11:48,anonymous,NaN,NaN,MBRF,Thalita Bento de Souza/ (11) 94274-2549 / tha...,4%,6%,"Vietnã (bovino); Indonésia (Suino); México, Ca...",0201.30.00; 0202.3000; 0203.2; 1602.50.00,02032


In [132]:
tarifas.query('SH6.str.startswith("02032")')

,SH6,Instrumento,Tarifa,Vigência,Tarifa Desc,Articles of Civil Aircraft only,ncm_2d,ncm_4d,ncm_5d,ncm_6d
3356,020321,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,NaN,02,0203,02032,020321
3357,020322,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,NaN,02,0203,02032,020322
3358,020329,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,NaN,02,0203,02032,020329


In [131]:
#Fazer o merge dos dataframes com tarifas, comparando os dígitos de ncm com os primeiros dígitos de tarifas, sendo que pode ser que o número de dígitos de ncm seja menor que o de tarifas
tarifas['ncm_2d'] = tarifas['SH6'].astype(str).str[:2]
tarifas['ncm_4d'] = tarifas['SH6'].astype(str).str[:4]
tarifas['ncm_5d'] = tarifas['SH6'].astype(str).str[:5]
tarifas['ncm_6d'] = tarifas['SH6'].astype(str).str[:6]


In [133]:
tarifas_2d = tarifas.merge(empresas_2d, left_on='ncm_2d', right_on='ncm', how='inner')
tarifas_4d = tarifas.merge(empresas_4d, left_on='ncm_4d', right_on='ncm', how='inner')
tarifas_5d = tarifas.merge(empresas_5d, left_on='ncm_5d', right_on='ncm', how='inner')
tarifas_6d = tarifas.merge(empresas_6d, left_on='ncm_6d', right_on='ncm', how='inner')

In [134]:
print(f'O tamanho dos dataframes de tarifas é:')
print(f'tarifas_2d: {tarifas_2d.shape[0]}')
print(f'tarifas_4d: {tarifas_4d.shape[0]}')
print(f'tarifas_5d: {tarifas_5d.shape[0]}')
print(f'tarifas_6d: {tarifas_6d.shape[0]}')

O tamanho dos dataframes de tarifas é:
tarifas_2d: 68
tarifas_4d: 97
tarifas_5d: 3
tarifas_6d: 540


In [135]:
tarifas_completas = pd.concat([tarifas_2d, tarifas_4d, tarifas_5d, tarifas_6d], ignore_index=True)

In [138]:
tarifas_completas.columns

Index(['SH6', 'Instrumento', 'Tarifa', 'Vigência', 'Tarifa Desc',
       'Articles of Civil Aircraft only', 'ncm_2d', 'ncm_4d', 'ncm_5d',
       'ncm_6d', 'ID', 'Start time', 'Completion time', 'Email', 'Name',
       'Last modified time', 'Empresa', 'Contato (Nome, Telefone e E-mail)',
       'Qual a participação (%) das exportações aos Estados Unidos na receita total da empresa?\n(Resposta numérica – ex: 25)',
       'Qual a participação (%) das exportações aos Estados Unidos em relação ao total exportado pela empresa?\n(Resposta numérica – ex: 25)\n',
       'Quais mercados a empresa entende que podem ser acessados no médio prazo?\n(Resposta longa – ex: América Latina, Oriente Médio, Sudeste Asiático etc.)\n',
       'Liste os Códigos Tarifários (NCM) mais Representativos nas Exportações para os EUA:',
       'ncm'],
      dtype='object')

In [143]:
tarifas_completas[['Empresa','Contato (Nome, Telefone e E-mail)','Qual a participação (%) das exportações aos Estados Unidos na receita total da empresa?\n(Resposta numérica – ex: 25)',
       'Qual a participação (%) das exportações aos Estados Unidos em relação ao total exportado pela empresa?\n(Resposta numérica – ex: 25)\n',
       'Quais mercados a empresa entende que podem ser acessados no médio prazo?\n(Resposta longa – ex: América Latina, Oriente Médio, Sudeste Asiático etc.)\n',
       'Liste os Códigos Tarifários (NCM) mais Representativos nas Exportações para os EUA:','Instrumento','Tarifa Desc','Articles of Civil Aircraft only']].fillna('').drop_duplicates().to_excel('empresas_tarifas.xlsx',index=False)

In [80]:
empresas_2d

,ID,Start time,Completion time,Email,Name,Last modified time,Empresa,"Contato (Nome, Telefone e E-mail)",Qual a participação (%) das exportações aos Estados Unidos na receita total da empresa?\n(Resposta numérica – ex: 25),Qual a participação (%) das exportações aos Estados Unidos em relação ao total exportado pela empresa?\n(Resposta numérica – ex: 25)\n,"Quais mercados a empresa entende que podem ser acessados no médio prazo?\n(Resposta longa – ex: América Latina, Oriente Médio, Sudeste Asiático etc.)\n",Liste os Códigos Tarifários (NCM) mais Representativos nas Exportações para os EUA:,ncm
11,12,2025-07-24 17:44:13,2025-07-24 18:17:14,anonymous,NaN,NaN,Bozel Brasil S/A,Fernando Neri de Araújo - Diretor Financeiro\n...,9%,17%,Infelizmente nenhum que possa ser substituído ...,Cálcio Silício (CaSi) - 7202.99.90\nFeSi 75% (...,75
11,12,2025-07-24 17:44:13,2025-07-24 18:17:14,anonymous,NaN,NaN,Bozel Brasil S/A,Fernando Neri de Araújo - Diretor Financeiro\n...,9%,17%,Infelizmente nenhum que possa ser substituído ...,Cálcio Silício (CaSi) - 7202.99.90\nFeSi 75% (...,75
36,37,2025-07-27 23:40:08,2025-07-27 23:44:31,anonymous,NaN,NaN,ABIMÓVEL - Associação Brasiliera das Indústria...,"Cândida Maria Cervieri, executiva@abimovel.com...",Entre 40 a 80%,28% das exportações brasileiras são destinadas...,"América Latina, Oriente Médio, União Europeia ...",Capitulo 94,94
132,133,2025-07-30 23:43:32,2025-07-30 23:47:59,anonymous,NaN,NaN,WEG EQUIPAMENTOS ELÉTRICOS S.A,"Ana Paula Hauffe Torquato, 47 3276-5288, anapa...",.,.,A empresa possui presença em diversos mercados...,"8501.51.10, 8501.52.10, 8501.53.10, 8501.40.19...",22
132,133,2025-07-30 23:43:32,2025-07-30 23:47:59,anonymous,NaN,NaN,WEG EQUIPAMENTOS ELÉTRICOS S.A,"Ana Paula Hauffe Torquato, 47 3276-5288, anapa...",.,.,A empresa possui presença em diversos mercados...,"8501.51.10, 8501.52.10, 8501.53.10, 8501.40.19...",13
132,133,2025-07-30 23:43:32,2025-07-30 23:47:59,anonymous,NaN,NaN,WEG EQUIPAMENTOS ELÉTRICOS S.A,"Ana Paula Hauffe Torquato, 47 3276-5288, anapa...",.,.,A empresa possui presença em diversos mercados...,"8501.51.10, 8501.52.10, 8501.53.10, 8501.40.19...",13
132,133,2025-07-30 23:43:32,2025-07-30 23:47:59,anonymous,NaN,NaN,WEG EQUIPAMENTOS ELÉTRICOS S.A,"Ana Paula Hauffe Torquato, 47 3276-5288, anapa...",.,.,A empresa possui presença em diversos mercados...,"8501.51.10, 8501.52.10, 8501.53.10, 8501.40.19...",13


In [32]:
#Pegar os 6 primeiros dígitos de ncm, de o número de dígitos for maior que 6 ou manter o total de dígitos, caso contrário, para cada item da lista ncm. Manter apenas valores únicos de códigos a 6 dígitos ou menos
empresas['ncm'].apply(lambda x: list(set([i[:6] if len(i) > 6 else i for i in x])))   


TypeError: 'float' object is not iterable